In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Parameters of the Axelrod model
L_values = [50, 100, 150]  # size of the grid
F_values = [2, 3, 10]  # number of features
q = 10  # number of traits per feature
# Parameter for the noise
noise = [0.01, 0.1]  # probability of random cultural drift
# Parameter for the mass media influence
media = [0.01, 0.1]  # probability of adopting the global culture

In [3]:
def initialize_culture_grid(L, F, q):
    """
    Initialize the culture grid.

    Parameters:
        L (int): size of the grid
        F (int): number of features
        q (int): number of traits per feature

    Returns:
        np.array: the culture grid
    """
    return np.random.randint(q, size=(L, L, F))

def simulation_step(culture_grid, L):
    """
    Perform a simulation step.

    Parameters:
        culture_grid (np.array): the culture grid
        L (int): size of the grid
        F (int): number of features
        q (int): number of traits per feature

    Returns:
        np.array: the updated culture grid
    """
    # Randomly select an agent
    x, y = np.random.randint(L, size=2)

    # Randomly select a neighbor
    dx, dy = np.random.choice([-1, 0, 1], size=2)
    nx, ny = (x + dx) % L, (y + dy) % L  # periodic boundary conditions

    # If they share at least one cultural feature
    if any(culture_grid[x, y, :] == culture_grid[nx, ny, :]):
        # The agent adopts one differing feature from the neighbor
        differing_features = np.where(culture_grid[x, y, :] != culture_grid[nx, ny, :])[0]
        if len(differing_features) > 0:
            f = np.random.choice(differing_features)
            culture_grid[x, y, f] = culture_grid[nx, ny, f]

    return culture_grid

def simulation_step_with_noise(culture_grid, L, noise):
    """
    Perform a simulation step with noise.

    Parameters:
        culture_grid (np.array): the culture grid
        L (int): size of the grid
        F (int): number of features
        q (int): number of traits per feature
        noise (float): probability of random cultural drift

    Returns:
        np.array: the updated culture grid
    """
    # Perform a normal simulation step
    simulation_step(culture_grid, L)

    # With a small probability, an agent randomly changes one of its features
    if np.random.rand() < noise:
        x, y = np.random.randint(L, size=2)
        f = np.random.randint(F)
        culture_grid[x, y, f] = np.random.randint(q)

    return culture_grid

def simulation_step_with_noise_and_media(culture_grid, L, media):
    """
    Perform a simulation step with noise and media influence.

    Parameters:
        culture_grid (np.array): the culture grid
        L (int): size of the grid
        F (int): number of features
        q (int): number of traits per feature
        noise (float): probability of random cultural drift
        media (float): probability of adopting the global culture

    Returns:
        np.array: the updated culture grid
    """
    # The global culture
    global_culture = np.random.randint(q, size=F)

    # Perform a simulation step with noise
    simulation_step_with_noise(culture_grid, L)

    # With a small probability, an agent adopts the global culture
    if np.random.rand() < media:
        x, y = np.random.randint(L, size=2)
        culture_grid[x, y, :] = global_culture

def calculate_Smax_norm(culture_grid):
    """
    Calculate the size of the largest homogeneous domain (Smax).

    Parameters:
        culture_grid (np.array): the culture grid

    Returns:
        int: Smax value
    """
    L, _, F = culture_grid.shape
    culture_int = np.sum(culture_grid * (q ** np.arange(F)), axis=2)
    unique_vals, counts = np.unique(culture_int, return_counts=True)
    max_count = np.max(counts)
    Smax_norm = max_count / (L ** 2)
    return Smax_norm


In [ ]:
F = 3
q = 100
q_values = range(3, q+1)
Smax_final_dict = {}

for L in L_values:
    Smax_final = []  # Svuota la lista all'inizio di ogni iterazione su L
    for n in q_values:
        Smax_norm = []
        culture_grid = initialize_culture_grid(L, F, n)
        for _ in range(5000000):
            culture_grid = simulation_step(culture_grid, L)
            T = calculate_Smax_norm(culture_grid) / (L ** 2)
            Smax_norm.append(T)
        SF = np.max(Smax_norm)
        Smax_final.append(SF)
    Smax_final_dict[L] = Smax_final

# Creazione dei grafici
for L, Smax_final in Smax_final_dict.items():
    plt.plot(list(q_values), Smax_final, label=f"L = {L}")  # Converto q_values in una lista

plt.xlabel('q')
plt.ylabel('Smax')
plt.title('Variazione di Smax in relazione a q per ogni L')
plt.legend()
plt.grid(True)
plt.show()